In [152]:
import pandas as pd
import numpy as np

In [153]:
# parse event_time as datetime64
df = pd.read_csv('order_brush_order.csv', parse_dates=['event_time'])

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   orderid     222750 non-null  int64         
 1   shopid      222750 non-null  int64         
 2   userid      222750 non-null  int64         
 3   event_time  222750 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 6.8 MB


In [155]:
# event_time is not sorted yet
df.head(7)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43


In [156]:
# sort df by event_time & reset the index
df.sort_values('event_time', inplace=True, ignore_index=True)
df.head(7)

,orderid,shopid,userid,event_time
0,31075200506751,6042309,97707522,2019-12-27 00:00:00
1,31075200506752,104804492,97707522,2019-12-27 00:00:00
2,31075200506753,8715449,97707522,2019-12-27 00:00:00
3,31075201870570,190969466,170182475,2019-12-27 00:00:02
4,31075205798264,2859407,12532131,2019-12-27 00:00:05
5,31075205798265,94479614,12532131,2019-12-27 00:00:05
6,31075205798263,65551316,12532131,2019-12-27 00:00:05


In [157]:
len(df['shopid'].unique())

18770

In [158]:
# create result dataframe. column1 = shopid , column2 = userid, with zeros on each userid 
uq_shop = np.array(df['shopid'].unique())
result = pd.DataFrame(data=uq_shop, columns=['shopid'])
result['userid'] = 0
result


,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,0
3,190969466,0
4,2859407,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [159]:
def check_within_1hour(idx):

  start = df['event_time'][idx]
  end = start + pd.DateOffset(hours=1)

  # get rows within 1 hour range
  df_1hour = df[ (df['event_time'] >= start) & (df['event_time'] <= end) ]

  # get unique shopid
  unique_shop_id = df_1hour['shopid'].unique()
  arr_unique_shop = np.array(unique_shop_id)

  for uq in arr_unique_shop:
    # get count of unique buyer of the particular shop
    unq_buy = df_1hour[df_1hour['shopid'] == uq]['userid'].nunique()

    # get number of orders within 1 hour of the particular shop
    num_ord = len(df_1hour[df_1hour['shopid'] == uq])
    
    # check if the particular shop deemed order brushing or not
    order_brush = num_ord / unq_buy
    if order_brush >= 3:
      #print(str(uq) + ' -- ' + str(order_brush))
      shop_idx = result[result['shopid'] == uq].index
      result.loc[ shop_idx , 'userid'] = 'orderbrushing'

In [ ]:
# main process , check each row
for x in range(len(df)):
  check_within_1hour(x)

In [ ]:
#show all shopid that deemed order brushing
result[result['userid'] == 'orderbrushing']['shopid']